In [1]:
%matplotlib inline

import sklearn
from sklearn.model_selection import cross_val_predict, cross_validate
from sklearn import linear_model
import matplotlib.pyplot as plt

import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from sklearn import metrics

## Regression

In [132]:
data = pd.read_csv("facebook_train.csv")
data.head()

,total_followers,type,category,month,weekday,hour,paid,comment,like,share
0,124050,Photo,3,6,4,17,1,3,77,16
1,132556,Photo,2,8,3,24,0,11,313,50
2,92786,Photo,2,2,6,17,1,2,145,39
3,135632,Photo,1,9,2,1,0,2,299,49
4,139540,Link,1,12,3,17,1,7,229,21


In [133]:
mapper = DataFrameMapper([
     (['category'], sklearn.preprocessing.OneHotEncoder()),
     (['month'], sklearn.preprocessing.OneHotEncoder()),
     (['weekday'], sklearn.preprocessing.OneHotEncoder()),
     (['hour'], None),#sklearn.preprocessing.OneHotEncoder()),
     ("type", sklearn.preprocessing.LabelBinarizer()),
     (["total_followers"], sklearn.preprocessing.StandardScaler()),
     ("paid", None),
#      ("comment", None),
#      ("share", None)
], df_out=True)
processed_data = mapper.fit_transform(data.copy())

pred_target = "like"

# Select features
exclude = [pred_target, "comment", "share"]
features = list(set(list(processed_data)) - set(exclude))

In [145]:
x, x_test, y, y_test = train_test_split(data, data.like, test_size=0.2)
lr = linear_model.LinearRegression()

# Pipeline will process data, then select the k best features, 
# then fit a linear model using these features. This will be evaluated using
# cross validating with 8 folds.
pipe = sklearn.pipeline.Pipeline([
    ('process', mapper),
    ('select', sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_regression)),
    ('lm', sklearn.linear_model.Lasso(alpha=0.02))])

for k in range(1, 15):
    pipe.set_params(select__k=k)
    cv_scores = np.round(cross_validate(pipe, X=x, y=y, cv=8)["test_score"], 2)
    print (k, np.average(cv_scores))

(1, -0.105)
(2, -0.1825)
(3, -0.24125000000000002)
(4, -0.23374999999999999)
(5, -0.2525)
(6, -0.22999999999999998)
(7, -0.21875)
(8, -0.23)
(9, -0.25125)
(10, -0.24125)
(11, -0.235)
(12, -0.23750000000000002)
(13, -0.24250000000000002)
(14, -0.2625)


In [135]:
pipe.set_params(select__k=5)
pipe.fit(x, y)

predictions = pipe.predict(x_test)
metrics_to_run = [metrics.mean_squared_error, metrics.mean_absolute_error, metrics.median_absolute_error, metrics.r2_score]
for metric in metrics_to_run: 
    print(metric.__name__, metric(predictions, y_test))
    
selected_features = pipe.named_steps['select'].get_support()
for selected, feature in zip(selected_features, features):
    print selected, feature

pipe.named_steps['lm'].coef_

('mean_squared_error', 563296.4505628401)
('mean_absolute_error', 261.09206281095646)
('median_absolute_error', 116.65238443948321)
('r2_score', -143.62935294490242)
True category_2
False category_1
True category_0
False type_Photo
False type_Status
False month_10
True month_11
False month_4
False month_5
False month_6
False month_7
True month_0
False month_1
False month_2
False month_3
False month_8
False month_9
False type_Video
False paid
False weekday_2
False weekday_3
False weekday_0
False weekday_1
False weekday_6
False weekday_4
False hour
False weekday_5
False type_Link
True total_followers


array([-23.4625078 ,  70.31020008,  84.84979447, 106.34890309,
        50.18687191])

In [143]:
test_data = pd.read_csv("facebook_test_2.csv")

mapper = DataFrameMapper([
     (['category'], sklearn.preprocessing.OneHotEncoder()),
     (['month'], sklearn.preprocessing.OneHotEncoder()),
     (['weekday'], sklearn.preprocessing.OneHotEncoder()),
     (['hour'], None),#sklearn.preprocessing.OneHotEncoder()),
     ("type", sklearn.preprocessing.LabelBinarizer()),
     (["total_followers"], sklearn.preprocessing.StandardScaler()),
     ("paid", None)
], df_out=True)

processed_test_data = mapper.fit_transform(test_data.copy())
predicted = pipe.named_steps['lm'].predict(processed_test_data[["category_2", "category_0", "month_0", "month_11", "total_followers"]])
# processed_test_data.head()
metrics_to_run = [metrics.mean_squared_error, metrics.mean_absolute_error, metrics.median_absolute_error, metrics.r2_score]
for metric in metrics_to_run: 
    print(metric.__name__, metric(predicted, test_data["like"]))

('mean_squared_error', 57605.907643400424)
('mean_absolute_error', 141.95153098379203)
('median_absolute_error', 107.15259130392067)
('r2_score', -8.810266155514672)


## Classifier

In [97]:
cas_data = pd.read_csv("casualty_train.csv")
cas_data.head()

,casualty_class,gender,age,severe,pedestrian_location,pedestrian_movement,travel,year
0,passenger,female,33,False,NaN,NaN,motorbike,2007
1,passenger,female,20,False,NaN,NaN,car,2005
2,passenger,male,52,False,NaN,NaN,car,2006
3,passenger,female,17,False,NaN,NaN,pedestrian,2012
4,passenger,female,20,False,NaN,NaN,motorbike,2010


In [119]:
cas_data = cas_data.loc[cas_data.age != 'Unknown']
cas_data.loc[np.logical_not(cas_data.pedestrian_location == cas_data.pedestrian_location), 'pedestrian_location'] = ''
cas_data.loc[np.logical_not(cas_data.pedestrian_movement == cas_data.pedestrian_movement), 'pedestrian_movement'] = ''
cas_data.severe = cas_data.severe.astype('int')

cas_mapper = DataFrameMapper([
     (['casualty_class'], sklearn.preprocessing.LabelBinarizer()),
     (['gender'], sklearn.preprocessing.LabelBinarizer()),
     (['travel'], sklearn.preprocessing.LabelBinarizer()),
     ("age", None),
     ("year", None),
], df_out=True)
processed_cas_data = cas_mapper.fit_transform(cas_data.copy())

pred_target = "severe"

# Select features
exclude = [pred_target]
features = list(set(list(processed_cas_data)) - set(exclude))
processed_cas_data.head()

,casualty_class_driver,casualty_class_passenger,casualty_class_pedestrian,gender,travel_bicycle,travel_bus,travel_car,travel_motorbike,travel_other,travel_pedestrian,travel_taxi,age,year
0,0,1,0,0,0,0,0,1,0,0,0,33,2007
1,0,1,0,0,0,0,1,0,0,0,0,20,2005
2,0,1,0,1,0,0,1,0,0,0,0,52,2006
3,0,1,0,0,0,0,0,0,0,1,0,17,2012
4,0,1,0,0,0,0,0,1,0,0,0,20,2010


In [121]:
x, x_test, y, y_test = train_test_split(cas_data, cas_data.severe, test_size=0.1)

pipe = sklearn.pipeline.Pipeline([
    ('process', cas_mapper),
    ('select', sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif)),
    ('svc', sklearn.svm.LinearSVC())])

for k in range(1, 10):
    pipe.set_params(select__k=k)
    cv_scores = np.round(cross_validate(pipe, X=x, y=y, cv=3)["test_score"], 2)
    print (k, np.average(cv_scores))

(1, 0.89)
(2, 0.89)
(3, 0.89)
(4, 0.89)
(5, 0.89)
(6, 0.89)


KeyboardInterrupt: 

In [ ]:
pipe.set_params(select__k=2)
pipe.fit(x, y)

predictions = pipe.predict(x_test)
metrics_to_run = [metrics.mean_squared_error, metrics.mean_absolute_error, metrics.median_absolute_error, metrics.r2_score]
for metric in metrics_to_run: 
    print(metric.__name__, metric(predictions, y_test))